In [1]:
using Pkg
Pkg.activate(".")

In [2]:
using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames

In [3]:
include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions.jl")

construct_ef_model (generic function with 1 method)

In [4]:
function second_func(scen)
    
    model = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_$(scen).mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    return model
end

second_func (generic function with 1 method)

In [5]:
function first_func()
    
    model = JuMP.read_from_file("./storage_expansion_revised/first_stage/first_stage_model.mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    
    return model
end

first_func (generic function with 1 method)

In [6]:
model1 = second_func(1)

Academic license - for non-commercial use only - expires 2022-08-19


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3456 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2304 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [7]:
model = JuMP.read_from_file("./ef_12scen_test_revised.mps")

A JuMP Model
Minimization problem with:
Variables: 52752
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 24768 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 41520 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 27673 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 17856 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 576 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [8]:
for scen = 1:12
    println(scen,"...")
    for ts in timesteps
        print(ts,".")
        for gen in gens
            con = get_thermal_interval(model,gen,ts,scen)
            conname = JuMP.name(con)
            var = get_thermal_variable(model,gen,ts,scen)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model, var >= lb, base_name = string("Tlb_{",string(gen),",",string(ts),"}_",string(scen)))
            JuMP.@constraint(model, var <= ub, base_name = string("Tub_{",string(gen),",",string(ts),"}_",string(scen)))
        end
        
        for branch in branches
            con = get_branch_interval(model,branch,ts,scen)
            conname = JuMP.name(con)
            var = get_line_variable(model,branch,ts,scen)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper
            coeff = 0
            for (key, value) in JuMP.constraint_object(con).func.terms
                coeff = value
            end

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model,var >= lb/coeff, base_name=string("Blb_{",string(branch),",",string(ts),"}_",string(scen)))
            JuMP.@constraint(model,var <= ub/coeff, base_name=string("Bub_{",string(branch),",",string(ts),"}_",string(scen)))
        end
    end
end
            

1...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.2...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.3...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.4...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.5...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.6...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.7...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.8...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.9...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.10...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.11...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.12...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.

In [12]:
JuMP.set_optimizer(model, Gurobi.Optimizer)
JuMP.set_optimizer_attribute(model, "Method", 1)  

JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2022-08-19
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 129673 rows, 52752 columns and 1439880 nonzeros
Model fingerprint: 0x5a2f6d24
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [5e-03, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 2e+02]
Presolve removed 86772 rows and 3800 columns
Presolve time: 0.52s
Presolved: 42901 rows, 49780 columns, 1007088 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          1s
    5941    3.2263239e+02   5.212213e+06   0.000000e+00      5s
    9103    5.4081643e+02   1.132073e+07   0.000000e+00     10s
   12696    5.9061290e+02   2.221129e+07   0.000000e+00     15s
   15959    6.1972526

In [13]:
for bus in buses
    varPR = get_PR_variable(model,bus,0)
    varER = get_ER_variable(model,bus,0)
    
    println(varPR, " ", JuMP.value(varPR))
    println(varER, " ", JuMP.value(varER))
end

PR_{101}_0 0.0
ER_{101}_0 0.0
PR_{102}_0 0.0
ER_{102}_0 0.0
PR_{103}_0 0.0
ER_{103}_0 0.0
PR_{104}_0 0.0
ER_{104}_0 0.0
PR_{105}_0 0.0
ER_{105}_0 0.0
PR_{106}_0 0.0
ER_{106}_0 0.0
PR_{107}_0 0.0
ER_{107}_0 0.0
PR_{108}_0 0.0
ER_{108}_0 0.0
PR_{109}_0 0.0
ER_{109}_0 0.0
PR_{110}_0 0.0
ER_{110}_0 0.0
PR_{111}_0 0.0
ER_{111}_0 0.0
PR_{112}_0 0.0
ER_{112}_0 0.0
PR_{113}_0 0.0
ER_{113}_0 0.0
PR_{114}_0 0.2970924254297301
ER_{114}_0 1.4854621271486503
PR_{115}_0 0.0
ER_{115}_0 0.0
PR_{116}_0 1.6240825552933416
ER_{116}_0 8.120412776466708
PR_{117}_0 1.4697830484937653
ER_{117}_0 7.348915242468826
PR_{118}_0 0.0
ER_{118}_0 0.0
PR_{119}_0 0.0
ER_{119}_0 0.0
PR_{120}_0 0.0
ER_{120}_0 0.0
PR_{121}_0 0.0
ER_{121}_0 0.0
PR_{122}_0 0.08005897876650214
ER_{122}_0 0.4002948938325107
PR_{123}_0 0.0
ER_{123}_0 0.0
PR_{124}_0 0.0
ER_{124}_0 0.0


In [14]:
model

A JuMP Model
Minimization problem with:
Variables: 52752
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 24768 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 59376 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 45529 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 576 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi

In [15]:
JuMP.write_to_file(model, "./ef_12scen_test_revised_noint.mps")

In [17]:
for scen = 1:12
    println(scen,"...")
    model = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_$(scen).mps")
    for ts in timesteps
        print(ts,".")
        for gen in gens
            con = get_thermal_interval(model,gen,ts)
            conname = JuMP.name(con)
            var = get_thermal_variable(model,gen,ts)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model, var >= lb, base_name = string("Tlb_{",string(gen),",",string(ts),"}"))
            JuMP.@constraint(model, var <= ub, base_name = string("Tub_{",string(gen),",",string(ts),"}"))
        end
        
        for branch in branches
            con = get_branch_interval(model,branch,ts)
            conname = JuMP.name(con)
            var = get_line_variable(model,branch,ts)
            lb = JuMP.constraint_object(con).set.lower
            ub = JuMP.constraint_object(con).set.upper
            coeff = 0
            for (key, value) in JuMP.constraint_object(con).func.terms
                coeff = value
            end

            JuMP.delete(model,con)
            JuMP.unregister(model, Symbol(conname))

            JuMP.@constraint(model,var >= lb/coeff, base_name=string("Blb_{",string(branch),",",string(ts),"}"))
            JuMP.@constraint(model,var <= ub/coeff, base_name=string("Bub_{",string(branch),",",string(ts),"}"))
        end
    end
    println("writing...")
    JuMP.write_to_file(model,"./storage_expansion_revised/second_stage/noint_scen_$(scen).mps")
end
            

1...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
2...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
3...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
4...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
5...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
6...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
7...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
8...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
9...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
10...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
11...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...
12...
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.writing...


# determine scenarios needed based on Yan et al.

In [1]:
function scen_conf(K, eps, S)
    
    sum = 0
    
    for i = 0:2*S
        #println("i=$(i)")
        sum += binomial(BigInt(K),BigInt(i))/BigFloat(eps^(-i))/BigFloat((1-eps)^-(K-i))
    end
    
    return sum
    
end

scen_conf (generic function with 1 method)

In [25]:
for K = 10:1000
    sum = scen_conf(K,.5,24)
    println(K, " ", sum)
end

10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
21 1.0
22 1.0
23 1.0
24 1.0
25 1.0
26 1.0
27 1.0
28 1.0
29 1.0
30 1.0
31 1.0
32 1.0
33 1.0
34 1.0
35 1.0
36 1.0
37 1.0
38 1.0
39 1.0
40 1.0
41 1.0
42 1.0
43 1.0
44 1.0
45 1.0
46 1.0
47 1.0
48 1.0
49 0.9999999999999982236431605997495353221893310546875
50 0.99999999999995470290059529361315071582794189453125
51 0.999999999999410693618528966908343136310577392578125
52 0.9999999999947866147209651899174787104129791259765625
53 0.99999999996473010188680063947685994207859039306640625
54 0.999999999805430583865728522141580469906330108642578125
55 0.9999999990885827527709039941328228451311588287353515625
56 0.99999999627239484489837906266984646208584308624267578125
57 0.999999986415737167344541802549429121427237987518310546875
58 0.9999999552029878550907238121681075426749885082244873046875
59 0.99999986468601484955465164006227496429346501827239990234375
60 0.999999621935950880162458087596633049543015658855438232421875
6

151 4.475798307470732999338399739554822982225924348747016162801627934450649745139719e-06
152 3.246549475714506860836159240135333848886965858573421336692133150599648982379235e-06
153 2.348252252508033913469137336713399482216188500369640502227502347016225348054265e-06
154 1.693778561314746480387449949934561586498907853678314465688985618691159557331787e-06
155 1.218358993183773533714903451991443492440128515987445552354402712266347615014515e-06
156 8.740130443038632218726384651634841252480219863515358254064571492016473764202299e-07
157 6.253187478905946633198915302321801378315006038367121337218297980993638707688014e-07
158 4.462132224920480408759407559926630459764829109246602089764789076266184102767176e-07
159 3.175828906149100120298306544933734981896974769241865539320905408325557613778574e-07
160 2.254557610272300724508599061222607139640268282481716388327313051557271074367819e-07
161 1.596506684646015441801665144286087252314049363367324137617604225294209260502993e-07
162 1.127718193381272

245 6.459851227133105713693407528130652317814473143101667947707274730912538869851725e-23
246 4.006638555157527275013484882321786686450965541239335913059762438420942374741085e-23
247 2.482673107415122487348684450834461066967574455233947830930247226418586976263263e-23
248 1.536895555675489365355705288579764011660042801356232111618713916457326213891348e-23
249 9.505134735969168297200582079818518373693731759520483656455632642813445412207607e-24
250 5.873066615631741397367842699995629532938091542464718662741341289783111171039045e-24
251 3.625499709482343563312929717431960695836829031910408641878537234690082556454538e-24
252 2.235994060606730616643513070576096464032846543858359835877148520827151073053772e-24
253 1.377769983360028502524167494576886203212739713002682632170408432852987509776828e-24
254 8.48182930815112319909059126948105237389600619816130601590395646859174384144958e-25
255 5.216899421102368092677059100119150303044614701331592040968926186008526998961837e-25
256 3.2058918819781348

342 1.445662567776089499725541478442761601372350322920917619556474817937843740259082e-44
343 8.398994764844111495742259408769930153605061591332015596898647805635700088063849e-45
344 4.877355098521303460116814842921478882383662469650886638691067418866617208601562e-45
345 2.830996914036428520496624081685216657214471088133473865678554491419717697562665e-45
346 1.642455544259859742434392073896478496131354881696592709635933346690457880545124e-45
347 9.524634067721201498009486599788553355025659430604167364970962391798473156322562e-46
348 5.520833537482980116006060769865221636995730305073179694750619978651619209420136e-46
349 3.198629229944811714444073788509689240538371412265206846022821379026443920216728e-46
350 1.852367928730823720847506585032810824651962353145634546677303586220486817339669e-46
351 1.072249625047221406677310357852500484320433924847869141427417448005114323950645e-46
352 6.203989243987982841133848203272712277917763698437178918519883481476956025322504e-47
353 3.588011503391848

448 1.685991507602117060763556543934169350896751653765260719962793485395160250796313e-70
449 9.435861716559651616905272541675437522534865092543034747833667866831792339828706e-71
450 5.279502666187803703142303076523132113671697388236054659485007172047758966396843e-71
451 2.953182302173709721558551510206543265427387105974685207051055290638785063356621e-71
452 1.65148194960502934228526124170929565927579413165721917349309958528463217220136e-71
453 9.233030395047279419987176756687561568246554876083396596710748590221605053669813e-72
454 5.160622416338186403569840516979359165649446163069292649039425121124078324336807e-72
455 2.883684950409703362370985033201620383901801636334805959085261506628663133305791e-72
456 1.610949634688008296344597754185808165603056845592556027968376439558130071980101e-72
457 8.997151935494139947416166276769719259655229919424751841089406667834204200628032e-73
458 5.023629446492457846773594457521086478306171592797772065493292387173540252874914e-73
459 2.8042717636110305

548 3.355755804618934854515031795896101041740645721520194623239616070700606664619467e-96
549 1.83824679067648140163963270812681908346028153702464278891096280128235344072522e-96
550 1.006797241121185048716884106145446034462357447914804508425503075822651824040648e-96
551 5.513218902691800745062150512751719538360046102550325420241058158596678706775852e-97
552 3.018517925361236721303913442836003649442348154731892482992848514067810333982139e-97
553 1.65237215206116594638868838073930209122551356425989349354740037539723955840463e-97
554 9.043735256899390765588671338685338123206526449222544428906352064023626882121916e-98
555 4.948960167871132999919491785657614541059362918461160693097578312806613185613904e-98
556 2.707737116184068838236656823201611988700708323221941784444847117613869002613956e-98
557 1.481241036717525852118884934298523977764279233701266869867171384811540493019497e-98
558 8.101621405850460453727170147317066672617340442484025010854765093980856994594636e-99
559 4.43042509171395327

644 1.335087272450326011163730406678310152376132731334874536639379848145401915766191e-121
645 7.211105167917111942905046178768106875039733692191355913297503705316032218064547e-122
646 3.894396563228614816579831490518800720579576334028444525318349492075662388885066e-122
647 2.102930209860078743330593456564744051866146857963861952881448470542820491184443e-122
648 1.135418748566720780565771246240641660666156147601771031039482726860225576483021e-122
649 6.129625594683074806727672526656263694181611640062419332448212252716243225442538e-123
650 3.308709898136733445408541147037420732285199457421001824388850234987439450016081e-123
651 1.785790294519522577586750950399341060165043378752462472031221238705277943348973e-123
652 9.637167771440629797824512671394821326772974357895942146640433974285887718396143e-124
653 5.200148455804871041463557427310152281060968109453971088665268804481240865547616e-124
654 2.805624808606399580386116291452888738209777960501403236716687931851129795373989e-124
655 1.5135

755 1.372243798824808840918589152103081842761328345015372159919435997815154556472141e-151
756 7.325637602018124851708276414224142416000706023722487444721042485122950810242649e-152
757 3.910396717844500240512600552115493358953312990966828531739525518505989204037887e-152
758 2.087168685884391756256516893711933883505586244989371693145020339571546371952552e-152
759 1.113924426570474692125452518310597205766588446784503887754432363774146606656296e-152
760 5.944501700168438540976903094993352237666592422574330971134645201523277445888847e-153
761 3.172026735415914405435475576056841659577082623132099223331726935002334080922327e-153
762 1.692466112571124176019608576119475215028613508562997989055415012966963884050293e-153
763 9.029526709690722468775282862369309357947665440660406077399040293598545773160557e-154
764 4.816947996807046790905302014954055616301335133589227881988306863582990521145013e-154
765 2.569454884626091331483049663400107391009577902721643984436770312932233389483089e-154
766 1.3704

853 1.796434044562231258595606062914088005035326988312426777730187885055322335115991e-178
854 9.516344071654841642551764589175838948935164664496206708447264679769134814384862e-179
855 5.040792642766367581964128845918925114809195770756526712912624169936869547383641e-179
856 2.669915027091246564738336677093236559556591431136064261746504197256208578804962e-179
857 1.414054111857395332841506171824183710982192102723245042564450548361006976637045e-179
858 7.488669645425557124857683825685729005815368342128704623178374599165151515085507e-180
859 3.965641420758073209640257756665271750730416364462646661131497871477657774590188e-180
860 2.099869999679547511014013001246916151397510230791164670133799283879903691306903e-180
861 1.111838335807914444007504079042614540913335800398631103472456435767792538829284e-180
862 5.886555174847434288214006460672740202326013695450201558122988759741100651003653e-181
863 3.11638570682179906038586666199495439872163360088132786719512747876963693875348e-181
864 1.64972

955 9.214559784868789117503629877791649069516698846850639851743686039944836239616967e-207
956 4.850539491543682824438572935871597353397409738916612668111102582125257686501398e-207
957 2.553180790872536560027672915785931361872057102801563115493949616444906862284141e-207
958 1.34384510520571204130312323524578705280178071514364099126148790678808852412357e-207
959 7.072826948382296847437174143900407626428110561456797852094558640126863439269522e-208
960 3.722314590849829339531190707014234781958012301418132667660427306309022106621117e-208
961 1.958887034253793809054357319179406969079013217187256254800358183247000352595602e-208
962 1.030818095650951451558061105757024971172814903963887014516389712676023953023593e-208
963 5.424142012658670161579314792185504186641919404404290226382837670035516902291203e-209
964 2.854016601222406165457321183679761049670313317666093252729198185431195650536734e-209
965 1.501614365073629695882211009771717171918878673771103142138527626938528790989057e-209
966 7.90017

In [26]:
function scen_conf_cautious(K, eps, S)
    sum = 0
    
    for i = 0:2*S
        summand = binomial(BigInt(K),BigInt(i))
        if i > 0
            for j = 1:i
                summand = BigFloat(summand)*BigFloat(eps)
            end
        end
        if K-i > 0
            for j = 1:K-i
                summand = BigFloat(summand)BigFloat(1-eps)
            end
        end
        sum += summand
    end
    return sum
end

scen_conf_cautious (generic function with 1 method)

In [44]:
for i = 48:133
    val1 = scen_conf(i,.5,24)
    val2 = scen_conf(i,.5,24)
    println(i, " ", val1, " ", val1-val2)
end

48 1.0 0.0
49 0.9999999999999982236431605997495353221893310546875 0.0
50 0.99999999999995470290059529361315071582794189453125 0.0
51 0.999999999999410693618528966908343136310577392578125 0.0
52 0.9999999999947866147209651899174787104129791259765625 0.0
53 0.99999999996473010188680063947685994207859039306640625 0.0
54 0.999999999805430583865728522141580469906330108642578125 0.0
55 0.9999999990885827527709039941328228451311588287353515625 0.0
56 0.99999999627239484489837906266984646208584308624267578125 0.0
57 0.999999986415737167344541802549429121427237987518310546875 0.0
58 0.9999999552029878550907238121681075426749885082244873046875 0.0
59 0.99999986468601484955465164006227496429346501827239990234375 0.0
60 0.999999621935950880162458087596633049543015658855438232421875 0.0
61 0.9999990150607909566819742064325282626668922603130340576171875 0.0
62 0.99999759123830036697776202370135933961137197911739349365234375 0.0
63 0.999994438488499775489863619082342438559862785041332244873046875 0.0

In [28]:
scen_conf_cautious(80,.5,6)

6.008243175410135281148838056440586452566776642925105988979339599609375e-11

In [15]:
function scen_conf_exp(K, eps, S)
    
    sum = 0
    
    for i = 0:2*S
        #println("i=$(i)")
        sum += binomial(BigInt(K),BigInt(i))
        println(i, " ", sum)
    end
    
    return sum
    
end

scen_conf_exp (generic function with 1 method)

In [23]:
testans = scen_conf_exp(46,.5,24)

0 1
1 47
2 1082
3 16262
4 179447
5 1550201
6 10917020
7 64441700
8 325374515
9 1427090845
10 5503441266
11 18844224462
12 57754842117


57754842117

In [24]:
for i = 1:46
    testans = BigFloat(testans/2)
    println(i, " ", testans)
end

1 2.88774210585e+10
2 1.443871052925e+10
3 7.219355264625e+09
4 3.6096776323125e+09
5 1.80483881615625e+09
6 9.02419408078125e+08
7 4.512097040390625e+08
8 2.2560485201953125e+08
9 1.12802426009765625e+08
10 5.64012130048828125e+07
11 2.820060650244140625e+07
12 1.4100303251220703125e+07
13 7.0501516256103515625e+06
14 3.52507581280517578125e+06
15 1.762537906402587890625e+06
16 881268.9532012939453125
17 440634.47660064697265625
18 220317.238300323486328125
19 110158.6191501617431640625
20 55079.30957508087158203125
21 27539.654787540435791015625
22 13769.8273937702178955078125
23 6884.91369688510894775390625
24 3442.456848442554473876953125
25 1721.2284242212772369384765625
26 860.61421211063861846923828125
27 430.307106055319309234619140625
28 215.1535530276596546173095703125
29 107.57677651382982730865478515625
30 53.788388256914913654327392578125
31 26.8941941284574568271636962890625
32 13.44709706422872841358184814453125
33 6.723548532114364206790924072265625
34 3.361774266057182

In [22]:
6.008243175410135281148838056440586452566776642925105988979339599609375e-11 - testans

0.0